# N-Gram Language Model Implementation - BAC Dataset

In [1]:
import xml.etree.ElementTree as ET
import xml.etree
import pandas as pd
import numpy as np
import os, re, json
from gensim.parsing.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk import FreqDist

from sklearn.model_selection import train_test_split

In [2]:
# Path to the dataset files
path_BAC = 'Datasets/BAC/blogs/'

In [3]:
def expand(text):
    """
    Replace all abbreviations with their corresponding expansion
    """
    text_proc = re.sub(r"won\'t", "will not", text)
    text_proc = re.sub(r"can\'t", "can not", text_proc)
    text_proc = re.sub(r"n\'t", " not", text_proc)
    text_proc = re.sub(r"\'re", " are", text_proc)
    text_proc = re.sub(r"\'s", " is", text_proc)
    text_proc = re.sub(r"\'d", " would", text_proc)
    text_proc = re.sub(r"\'ll", " will", text_proc)
    text_proc = re.sub(r"\'t", " not", text_proc)
    text_proc = re.sub(r"\'ve", " have", text_proc)
    text_proc = re.sub(r"\'m", " am", text_proc)
    return text_proc

In [4]:
def replace_numbers(text):
    """
    Replace number appearances with 'NUM'
    """
    # Case 1: Combination of numbers and letters (Eg. 2nd -> NUM)
    text_proc = re.sub('[a-zA-Z]+[0-9]+[a-zA-Z]+', 'NUM', text)
    text_proc = re.sub('[0-9]+[a-zA-Z]+|[a-zA-Z]+[0-9]+', 'NUM', text_proc)
    # Case 2: Decimal numbers (Eg. 2.1 -> NUM)
    text_proc = re.sub('[0-9]+\.+[0-9]+', 'NUM', text_proc)
    # Case 3: Numbers between spaces (Eg. 220 888 -> NUM)
    text_proc = re.sub('([0-9]+\s)*[0-9]+', 'NUM', text_proc)
    # Case 4: One or more of the previous cases (Eg. NUM NUM -> NUM)
    text_proc = re.sub('((NUM)+\s)*(NUM)+', 'NUM', text_proc)
    return text_proc

In [5]:
def preprocess_BAC(text):
    """
    Removes dates, XML tags and performs text preprocessing:
    * Returns a string with no special characters
    * Keeps sentence dividers (dots, question marks and exclamation marks)
    """
    text_proc = re.sub('<date>[a-zA-Z0-9,]*<\/date>',' ', text)
    text_proc = re.sub('<Blog>',' ', text_proc)
    text_proc = re.sub('<\/Blog>',' ', text_proc)
    text_proc = re.sub('<post>',' ', text_proc)
    text_proc = re.sub('<\/post>',' ', text_proc)
    text_proc = text.lower()
    text_proc = expand(text_proc)
    text_proc = re.sub('[^a-z0-9 .!?]', ' ', text_proc)
    text_proc = re.sub('\.{2,}', ' ', text_proc)
    text_proc = re.sub('\s+', ' ', text_proc)
    text_proc = replace_numbers(text_proc)
    return text_proc

In [6]:
def remove_punctuation(text):
    """
    Removes all characters that are not letters or spaces
    """
    text_proc = re.sub('[^a-z\s]',' ',text)
    text_proc = re.sub('\s+', ' ', text_proc)
    return text_proc

In [7]:
def tokenize_sentence_BAC(path):
    """
    Returns a list of sentence tokens
    * Each post is separated into sentences using nltk.sentence_tokenize()
    """
    out_list = []
    list_BAC = os.listdir(path)
    for i, file in enumerate(list_BAC):
        with open(path+file, 'r',encoding="mbcs") as f:
            xml_string = f.read()
            xml_string = preprocess_BAC(xml_string)
            out_list += sent_tokenize(xml_string)
    return out_list

In [8]:
def get_inverted_index_BAC(sentence_list):
    """
    Returns a modified inverted index to get words that show up once
    * Each word has a list of at most two documents
    * Returns a dictionary with the modified inverted index for all words in the vocabulary
    * Reduces complexity of replacing words that show up once
    """
    out_idx = {}
    for i, sentence in enumerate(sentence_list):
        for word in sentence.split(' '):
            if word != '':
                if word not in out_idx:
                    out_idx[word] = [i]
                elif i not in out_idx[word] and len(out_idx[word]) == 1:
                    out_idx[word].append(i)
    return out_idx

In [9]:
def add_UNK_tags_BAC(sentences_list, least_frequent_words):
    """
    Returns a list with tagged sentences, including the tag for unknown words
    """
    # Replaces words that happen once with the tag '<UNK>'
    # Reduced complexity as it doesn't iterate over all words in all sentences
    for (index, word) in least_frequent_words_BAC:
        sentences_list[index] = sentences_list[index].replace(word, '<UNK>')
    # Adds sentence beginning and end tags
    for i, sentence in enumerate(sentences_list):
        sentences_list[i] = '<s> ' + sentence.strip() + ' </s>'
    return sentences_list

In [10]:
# Get list of processed sentences
out_list_sentences_raw_BAC = tokenize_sentence_BAC(path_BAC)
out_list_sentences_BAC = [remove_punctuation(sentence) for sentence in out_list_sentences_raw_BAC]
out_list_sentences_BAC

[' blog date may date post well everyone got up and going this morning ',
 'it is still raining but that is okay with me ',
 'sort of suits my mood ',
 'i could easily have stayed home in bed with my book and the cats ',
 'this has been a lot of rain though ',
 'people have wet basements there are lakes where there should be golf courses and fields everything is green green green ',
 'but it is supposed to be degrees by friday so we will be dealing with mosquitos next week ',
 'i heard winnipeg described as an old testament city on urllink cbc radio one last week and it sort of rings true ',
 'floods infestations etc ',
 'etc post date may date post my four year old never stops talking ',
 'she will say mom ',
 'and when i say yes ',
 'she will say ummm ummm oh yeah ',
 'where do lady bugs hide in the rain ',
 'anything to hear her own voice ',
 'very very exhausting ',
 'now i remember ',
 'this is why i go to work ',
 'sigh post date may date post actually it is not raining yet but i

In [11]:
# Get modified inverted index
inv_BAC = get_inverted_index_BAC(out_list_sentences_BAC)
# Get list of tuples (sentence_index, word) that contains words that appear once
least_frequent_words_BAC = [(item[1][0], item[0]) for item in inv_BAC.items() if len(item[1]) == 1]
least_frequent_words_BAC

[(82, 'ineveitable'),
 (123, 'poopeyhead'),
 (177, 'momly'),
 (446, 'jusat'),
 (497, 'mehgan'),
 (520, 'vnted'),
 (598, 'weieners'),
 (611, 'gianopolis'),
 (644, 'opur'),
 (730, 'kordalewski'),
 (763, 'malevalent'),
 (805, 'zimnie'),
 (820, 'albinak'),
 (853, 'evereryone'),
 (988, 'snippity'),
 (1019, 'lookis'),
 (1078, 'chemisry'),
 (1089, 'xocatiexo'),
 (1109, 'hahahahahahahahahahahahaahahahahahahahahaha'),
 (1182, 'lunh'),
 (1273, 'metranomes'),
 (1363, 'tyhough'),
 (1372, 'jerkyest'),
 (1584, 'drudgeries'),
 (1662, 'cleanng'),
 (1750, 'loweered'),
 (1997, 'highter'),
 (2027, 'overweis'),
 (2086, 'hwlp'),
 (2174,
  'ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh'),
 (2380, 'trustworhy'),
 (2547, 'creeeptacular'),
 (2548, 'creeptacular'),
 (2648, 'cloass'),
 (2669, 'loahte'),
 (2699,
  'ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh'),
 (2747, 'togetther'),
 (2885, 'distanceing'),
 (2949, 'ooowwwwwwwwwwwww

In [12]:
# Get tagged sentences (unknown words and beginning and ending tags)
tagged_sentences_BAC = add_UNK_tags_BAC(out_list_sentences_BAC, least_frequent_words_BAC)
tagged_sentences_BAC

['<s> blog date may date post well everyone got up and going this morning </s>',
 '<s> it is still raining but that is okay with me </s>',
 '<s> sort of suits my mood </s>',
 '<s> i could easily have stayed home in bed with my book and the cats </s>',
 '<s> this has been a lot of rain though </s>',
 '<s> people have wet basements there are lakes where there should be golf courses and fields everything is green green green </s>',
 '<s> but it is supposed to be degrees by friday so we will be dealing with mosquitos next week </s>',
 '<s> i heard winnipeg described as an old testament city on urllink cbc radio one last week and it sort of rings true </s>',
 '<s> floods infestations etc </s>',
 '<s> etc post date may date post my four year old never stops talking </s>',
 '<s> she will say mom </s>',
 '<s> and when i say yes </s>',
 '<s> she will say ummm ummm oh yeah </s>',
 '<s> where do lady bugs hide in the rain </s>',
 '<s> anything to hear her own voice </s>',
 '<s> very very exhausti

In [13]:
# Divide in train and test sets
train_BAC, test_BAC = train_test_split(tagged_sentences_BAC, test_size=0.2, random_state=0)
print("Train: {} - Test: {}".format(len(train_BAC),len(test_BAC)))

Train: 7098164 - Test: 1774541


In [14]:
# Save list of training sentences
with open("BAC_G02_training", "w") as out_file_training:
    for sentence in train_BAC:
        out_file_training.write(sentence+"\n")

In [15]:
# Save list of test sentences
with open("BAC_G02_testing", "w") as out_file_testing:
    for sentence in test_BAC:
        out_file_testing.write(sentence+"\n")

# N-gram models

In [10]:
# Path of training file
training_file = "BAC_G02_training"

In [11]:
def read_sentences(file_path):
    """
    Returns a list of sentences from file
    """
    file = open(file_path, 'r')
    return file.read().splitlines()

## Unigrams

In [12]:
def get_unigram_frequencies(sentence_list):
    """
    Returns the total word count and a dictionary with the frequencies of each word
    """
    word_count = 0
    unigram_frequencies = {}
    for sentence in sentence_list:
        for word in sentence.split(' '):
            if word != '':
                word_count += 1
                if word not in unigram_frequencies:
                    unigram_frequencies[word] = 0
                unigram_frequencies[word] += 1
    return word_count, unigram_frequencies

In [13]:
# Retrieve sentences from file
sentences = read_sentences(training_file)
sentences

['<s> this cop show is interesting to me because to me it is one of the first few shows that i am aware of that focuses mainly on the cop work not the personal life of said cops </s>',
 '<s> if i think of this as a one week effort then my brain will make a little contest of it get psyched into trying to see how well i can eat how much water i can drink how many hours i can exercise </s>',
 '<s> i gave him fair warning if he pulls this shit again he will find himself locked up at attention while i unload on him with both barrels and i do not care who witnesses it </s>',
 '<s> i guess they wanted to come at five and he said they had to be here by four </s>',
 '<s> rajiv gandhi s government was found involved in many cases of alleged corruptions which actually led to his defeat in the election </s>',
 '<s> i am addicted </s>',
 '<s> wad so boring besides the fact tt some ppl also came down we did the items thingy soo boring k not bad at least i passed all haa good enough for me i have bet

In [50]:
# Get word frequencies
total_words, unigram_frequencies = get_unigram_frequencies(sentences)
unigram_frequencies

{'<s>': 7098164,
 'this': 698395,
 'cop': 2146,
 'show': 51229,
 'is': 2069442,
 'interesting': 31683,
 'to': 3168543,
 'me': 758434,
 'because': 197000,
 'it': 1619512,
 'one': 370494,
 'of': 1940160,
 'the': 4254428,
 'first': 129116,
 'few': 76462,
 'shows': 11739,
 'that': 1494355,
 'i': 4392191,
 'am': 626105,
 'aware': 4729,
 'focuses': 470,
 'mainly': 3891,
 'on': 788009,
 'work': 135156,
 'not': 1266990,
 'personal': 14026,
 'life': 139517,
 'said': 123894,
 'cops': 2218,
 '</s>': 7098164,
 'if': 354815,
 'think': 238898,
 'as': 472755,
 'a': 2337890,
 'week': 75564,
 'effort': 7239,
 'then': 274082,
 'my': 1161417,
 'brain': 10251,
 'will': 493262,
 'make': 132904,
 'little': 122750,
 'contest': 2687,
 'get': 314924,
 'psyched': 425,
 'into': 145222,
 'trying': 50173,
 'see': 185149,
 'how': 216477,
 'well': 238309,
 'can': 380479,
 'eat': 28656,
 'much': 177220,
 'water': 25921,
 'drink': 15961,
 'many': 80117,
 'hours': 43523,
 'exercise': 4661,
 'gave': 27046,
 'him': 20867

In [18]:
# Calculate probabilities for each unigram as frequency_i/total_words
unigram_probabilities = {item[0]: item[1]/total_words for item in unigram_frequencies.items()}
unigram_probabilities

{'<s>': 0.055008963432966074,
 'this': 0.005412383401787609,
 'cop': 1.6630953515182967e-05,
 'show': 0.00039701170439389945,
 'is': 0.016037648510888756,
 'interesting': 0.0002455351818366924,
 'to': 0.024555401371788625,
 'me': 0.0058776703626907175,
 'because': 0.001526699833406824,
 'it': 0.01255080558680382,
 'one': 0.00287123415268136,
 'of': 0.015035745932906517,
 'the': 0.03297073359817933,
 'first': 0.0010006161202545964,
 'few': 0.0005925610287408761,
 'shows': 9.097426063128279e-05,
 'that': 0.011580870708378958,
 'i': 0.03403836176645154,
 'am': 0.00485215431063543,
 'aware': 3.664854574711102e-05,
 'focuses': 3.642380313204098e-06,
 'mainly': 3.0154259146121587e-05,
 'on': 0.006106869081335422,
 'work': 0.0010474245821519427,
 'not': 0.009818849857503108,
 'personal': 0.00010869792824042698,
 'life': 0.0010812212216112684,
 'said': 0.0009601469500512948,
 'cops': 1.7188935180184447e-05,
 '</s>': 0.055008963432966074,
 'if': 0.0027497258953819406,
 'think': 0.00185139866396

In [19]:
# Save unigram probability estimates in JSON format
with open("BAC_G02_unigrams.json", "w") as out_file_unigrams:
    json.dump(unigram_probabilities, out_file_unigrams)

## Bigrams

In [15]:
# Get all words as an array. Used to calculate the Laplace smoothing matrix
vocab_array = list(unigram_frequencies.keys())
vocab_array

['<s>',
 'this',
 'cop',
 'show',
 'is',
 'interesting',
 'to',
 'me',
 'because',
 'it',
 'one',
 'of',
 'the',
 'first',
 'few',
 'shows',
 'that',
 'i',
 'am',
 'aware',
 'focuses',
 'mainly',
 'on',
 'work',
 'not',
 'personal',
 'life',
 'said',
 'cops',
 '</s>',
 'if',
 'think',
 'as',
 'a',
 'week',
 'effort',
 'then',
 'my',
 'brain',
 'will',
 'make',
 'little',
 'contest',
 'get',
 'psyched',
 'into',
 'trying',
 'see',
 'how',
 'well',
 'can',
 'eat',
 'much',
 'water',
 'drink',
 'many',
 'hours',
 'exercise',
 'gave',
 'him',
 'fair',
 'warning',
 'he',
 'pulls',
 'shit',
 'again',
 'find',
 'himself',
 'locked',
 'up',
 'at',
 'attention',
 'while',
 'unload',
 'with',
 'both',
 'barrels',
 'and',
 'do',
 'care',
 'who',
 'witnesses',
 'guess',
 'they',
 'wanted',
 'come',
 'five',
 'had',
 'be',
 'here',
 'by',
 'four',
 'rajiv',
 'gandhi',
 's',
 'government',
 'was',
 'found',
 'involved',
 'in',
 'cases',
 'alleged',
 'corruptions',
 'which',
 'actually',
 'led',
 'hi

In [16]:
# Dictionary of unique indexes for each word in the vocabulary
vocab_indexes = {word: i for i,word in enumerate(vocab_array)}
vocab_indexes

{'<s>': 0,
 'this': 1,
 'cop': 2,
 'show': 3,
 'is': 4,
 'interesting': 5,
 'to': 6,
 'me': 7,
 'because': 8,
 'it': 9,
 'one': 10,
 'of': 11,
 'the': 12,
 'first': 13,
 'few': 14,
 'shows': 15,
 'that': 16,
 'i': 17,
 'am': 18,
 'aware': 19,
 'focuses': 20,
 'mainly': 21,
 'on': 22,
 'work': 23,
 'not': 24,
 'personal': 25,
 'life': 26,
 'said': 27,
 'cops': 28,
 '</s>': 29,
 'if': 30,
 'think': 31,
 'as': 32,
 'a': 33,
 'week': 34,
 'effort': 35,
 'then': 36,
 'my': 37,
 'brain': 38,
 'will': 39,
 'make': 40,
 'little': 41,
 'contest': 42,
 'get': 43,
 'psyched': 44,
 'into': 45,
 'trying': 46,
 'see': 47,
 'how': 48,
 'well': 49,
 'can': 50,
 'eat': 51,
 'much': 52,
 'water': 53,
 'drink': 54,
 'many': 55,
 'hours': 56,
 'exercise': 57,
 'gave': 58,
 'him': 59,
 'fair': 60,
 'warning': 61,
 'he': 62,
 'pulls': 63,
 'shit': 64,
 'again': 65,
 'find': 66,
 'himself': 67,
 'locked': 68,
 'up': 69,
 'at': 70,
 'attention': 71,
 'while': 72,
 'unload': 73,
 'with': 74,
 'both': 75,
 'bar

In [17]:
# Get the size of the vocabulary
vocab_len = len(vocab_array)
vocab_len

295114

In [54]:
def get_bigram_frequencies(sentence_list):
    """
    Returns a dictionary with the frequencies of each bigram in the data
    """
    bigram_frequencies = {}
    for sentence in sentence_list:
        word_list = sentence.split(' ')
        for i in range(1,len(word_list)):
            if word_list[i-1] != '' and word_list[i] != '':
                bigram = word_list[i-1] + ' ' + word_list[i]
                if bigram not in bigram_frequencies:
                    bigram_frequencies[bigram] = 0
                bigram_frequencies[bigram] += 1
    return bigram_frequencies

In [55]:
# Get the bigram frequencies
bigram_frequencies = get_bigram_frequencies(sentences)
bigram_frequencies

{'<s> this': 86326,
 'this cop': 36,
 'cop show': 21,
 'show is': 1570,
 'is interesting': 1402,
 'interesting to': 2398,
 'to me': 60282,
 'me because': 4008,
 'because to': 246,
 'me it': 5240,
 'it is': 380602,
 'is one': 15297,
 'one of': 76021,
 'of the': 363253,
 'the first': 53136,
 'first few': 964,
 'few shows': 41,
 'shows that': 987,
 'that i': 190525,
 'i am': 582819,
 'am aware': 487,
 'aware of': 2633,
 'of that': 16507,
 'that focuses': 37,
 'focuses mainly': 5,
 'mainly on': 88,
 'on the': 194909,
 'the cop': 438,
 'cop work': 1,
 'work not': 185,
 'not the': 22194,
 'the personal': 499,
 'personal life': 491,
 'life of': 3435,
 'of said': 428,
 'said cops': 1,
 'cops </s>': 311,
 '<s> if': 73623,
 'if i': 74970,
 'i think': 124325,
 'think of': 17065,
 'of this': 30598,
 'this as': 2025,
 'as a': 47465,
 'a one': 2081,
 'one week': 1394,
 'week effort': 3,
 'effort then': 9,
 'then my': 3256,
 'my brain': 3569,
 'brain will': 71,
 'will make': 6340,
 'make a': 14702,
 

In [45]:
# Calculate the probabilities of bigrams that appear in the data using Laplace smoothing
# For a bigram (w1w2) returns (count(w1w2)+1)/(count(w1)+V)
# Does not take into account bigram probabilities generated with the smoothing (bigrams that don't appear in the dataset)
bigram_probabilities = {item[0]: (item[1]+1)/
                        (unigram_frequencies[item[0].split(' ')[0]]+vocab_len)
                        for item in bigram_frequencies.items()}
bigram_probabilities

{'<s> this': 0.011676417415928361,
 'this cop': 3.7241736109084065e-05,
 'cop show': 7.400928480118414e-05,
 'show is': 0.004535965791137687,
 'is interesting': 0.0005933460658153159,
 'interesting to': 0.0073409486623194214,
 'to me': 0.01740443698668777,
 'me because': 0.0038052371605280444,
 'because to': 0.0005019162226638543,
 'me it': 0.004974619096614488,
 'it is': 0.19878712604968282,
 'is one': 0.006469713552988384,
 'one of': 0.11421437242340837,
 'of the': 0.16250983100953173,
 'the first': 0.011679637203041536,
 'first few': 0.002274709473634585,
 'few shows': 0.00011303205804465304,
 'shows that': 0.0032197827624302192,
 'that i': 0.10647069046739563,
 'i am': 0.12434010588173801,
 'am aware': 0.000529732886534038,
 'aware of': 0.008784597272572647,
 'of that': 0.007385224361756098,
 'that focuses': 2.1235349704297755e-05,
 'focuses mainly': 2.029879831113998e-05,
 'mainly on': 0.000297653885386532,
 'on the': 0.1799518614229409,
 'the cop': 9.649322942836884e-05,
 'cop wo

In [46]:
# Saves bigram probabilities in JSON format
with open("BAC_G02_bigrams.json", "w") as out_file_bigrams:
    json.dump(bigram_probabilities, out_file_bigrams)

## Trigrams

In [48]:
def get_trigram_frequencies(sentence_list):
    """
    Returns a dictionary of trigram frequencies for each trigram in the data
    """
    trigram_frequencies = {}
    for sentence in sentence_list:
        word_list = sentence.split(' ')
        for i in range(2,len(word_list)):
            if word_list[i-2] != '' and word_list[i-1] != '' and word_list[i] != '':
                trigram = word_list[i-2] + ' ' + word_list[i-1] + ' ' + word_list[i]
                if trigram not in trigram_frequencies:
                    trigram_frequencies[trigram] = 0
                trigram_frequencies[trigram] += 1
    return trigram_frequencies

In [49]:
# Get trigram frequencies
trigram_frequencies = get_trigram_frequencies(sentences)
trigram_frequencies

{'<s> this cop': 3,
 'this cop show': 1,
 'cop show is': 2,
 'show is interesting': 3,
 'is interesting to': 275,
 'interesting to me': 199,
 'to me because': 530,
 'me because to': 2,
 'because to me': 46,
 'to me it': 973,
 'me it is': 2142,
 'it is one': 2270,
 'is one of': 8249,
 'one of the': 31253,
 'of the first': 1927,
 'the first few': 778,
 'first few shows': 2,
 'few shows that': 8,
 'shows that i': 90,
 'that i am': 27692,
 'i am aware': 481,
 'am aware of': 212,
 'aware of that': 65,
 'of that focuses': 1,
 'that focuses mainly': 1,
 'focuses mainly on': 5,
 'mainly on the': 26,
 'on the cop': 12,
 'the cop work': 1,
 'cop work not': 1,
 'work not the': 5,
 'not the personal': 4,
 'the personal life': 9,
 'personal life of': 7,
 'life of said': 1,
 'of said cops': 1,
 'said cops </s>': 1,
 '<s> if i': 11832,
 'if i think': 312,
 'i think of': 2279,
 'think of this': 378,
 'of this as': 120,
 'this as a': 556,
 'as a one': 37,
 'a one week': 43,
 'one week effort': 1,
 'wee

In [51]:
# Calculate the probabilities of trigrams that appear in the data using Laplace smoothing
# For a trigram (w1w2w3) returns (count(w1w2w3)+1)/(count(w1w2)+V)
# Does not take into account trigram probabilities generated with the smoothing (trigrams that don't appear in the dataset)
trigram_probabilities = {item[0]: (item[1]+1)/
                         (bigram_frequencies[' '.join(item[0].split(' ')[:-1])]+vocab_len) 
                         for item in trigram_frequencies.items()}
trigram_probabilities

{'<s> this cop': 1.0486577181208053e-05,
 'this cop show': 6.7762154836523805e-06,
 'cop show is': 1.0164839819065852e-05,
 'show is interesting': 1.3482358334119805e-05,
 'is interesting to': 0.000930809804529941,
 'interesting to me': 0.0006722417919277205,
 'to me because': 0.0014941079809564542,
 'me because to': 1.0029352571860311e-05,
 'because to me': 0.00015912784398699892,
 'to me it': 0.0027406048464248332,
 'me it is': 0.007134914134654441,
 'it is one': 0.0033608794227160525,
 'is one of': 0.026577666384245403,
 'one of the': 0.08421194444070217,
 'of the first': 0.00292845783582713,
 'the first few': 0.0022368987796123475,
 'first few shows': 1.0132465093657752e-05,
 'few shows that': 3.049245311785333e-05,
 'shows that i': 0.0003073275672827853,
 'that i am': 0.057023838694997726,
 'i am aware': 0.0005490168384147766,
 'am aware of': 0.0007205658979502776,
 'aware of that': 0.00022166470191135426,
 'of that focuses': 6.418052698630708e-06,
 'that focuses mainly': 6.776192

In [52]:
# Saves trigram probabilities in JSON format
with open("BAC_G02_trigrams.json", "w") as out_file_trigrams:
    json.dump(trigram_probabilities, out_file_trigrams)

# Perplexity

In [99]:
def get_sentence_probability_unigrams(sentence, unigram_probabilities):
    """
    Calculates the probability of a sentence as the product of the probabilities of its unigrams
    """
    sentence_probability = 1
    for word in sentence.split(' '):
        if word in unigram_probabilities:
            sentence_probability *= unigram_probabilities[word]
    return sentence_probability

In [100]:
def get_sentence_probability_bigrams(sentence, bigram_probabilities, unigram_frequencies):
    """
    Calculates the probability of a sentence as the product of the probabilities of its bigrams
    using Laplace smoothing
    """
    sentence_probability = 1
    training_vocab_len = len(unigram_frequencies)
    word_list = sentence.split(' ')
    for i in range(1,len(word_list)):
        bigram = word_list[i-1] + ' ' + word_list[i]
        if word_list[i-1] in unigram_frequencies and word_list[i] in unigram_frequencies:
            # Case 1: the bigram appears in the training data (the probability is already calculated)
            if bigram in bigram_probabilities:
                sentence_probability *= bigram_probabilities[bigram]
            # Case 2: both words are in the training vocabulary but they don't appear as a bigram
            # The probability estimate is calculated dynamically for space reasons
            else:
                # The probability for the bigram (w1w2) is calculated as 1/(count(w1)+V)
                # Following Laplace smoothing
                bigram_probability = 1/(unigram_frequencies[word_list[i-1]]+training_vocab_len)
                sentence_probability *= bigram_probability
    return sentence_probability

In [101]:
def get_sentence_probability_trigrams(sentence, trigram_probabilities, bigram_frequencies, training_vocab_len):
    """
    Calculates the probability of a sentence as the product of the probabilities of its trigrams
    using Laplace smoothing
    """
    sentence_probability = 1
    word_list = sentence.split(' ')
    for i in range(2,len(word_list)):
        prev_bigram = word_list[i-2] + ' ' + word_list[i-1]
        if prev_bigram in bigram_frequencies and word_list[i] != '':
            trigram = prev_bigram + ' ' + word_list[i]
            # Case 1: the trigram appears in the training data (the probability is already calculated)
            if trigram in trigram_probabilities:
                sentence_probability *= trigram_probabilities[trigram]
            # Case 2: all three words are in the training vocabulary but they don't appear as a trigram
            # The probability estimate is calculated dynamically for space reasons
            else:
                # The probability for the trigram (w1w2w3) is calculated as 1/(count(w1w2)+V)
                # Following Laplace smoothing
                trigram_probability = 1/(bigram_frequencies[prev_bigram]+training_vocab_len)
                sentence_probability *= trigram_probability
    return sentence_probability

In [45]:
def get_perplexity(probability, number_of_words):
    """
    Calculates the perplexity as the Nth root of the inverse of the probability
    """
    # The Nth root is equivalent to the power of 1/N
    return np.power(1/probability, 1/number_of_words)

In [46]:
# Path to the test file
testing_file = "BAC_G02_testing"

In [47]:
# Get the test sentences
test_sentences = read_sentences(testing_file)
test_sentences

['<s> pretty much </s>',
 '<s> they could hit you and beat you and hit you </s>',
 '<s> i am not really gullible but i am intimidated by nearly everyone i know to some extent </s>',
 '<s> weeee alll <UNK> <UNK> <UNK> lol someone who didnt know me might be disturbed by my oddity i think i made another word but then that might already be a real word </s>',
 '<s> that is not something that is done too much anymore </s>',
 '<s> eeesh its a classic </s>',
 '<s> but in truth people who skip breakfast are more likely to be overweight or obese </s>',
 '<s> so i play nl is and nl after i get these lousy cards </s>',
 '<s> good thing i fixed her not that there was anything wrong with her </s>',
 '<s> i covet the prayers that come across this blog pray for the family i have mentioned and that god would make a way where there seems to be no way </s>',
 '<s> heavens know what will happen if i get really intoxicated </s>',
 '<s> we did a ton of walking and the heat took a lot out of us </s>',
 '<s> 

## Unigrams

In [48]:
# Retrieve unigram probabilities
unigram_file = open('BAC_G02_unigrams.json')
unigram_probabilities = json.load(unigram_file)
unigram_probabilities

{'<s>': 0.055008963432966074,
 'this': 0.005412383401787609,
 'cop': 1.6630953515182967e-05,
 'show': 0.00039701170439389945,
 'is': 0.016037648510888756,
 'interesting': 0.0002455351818366924,
 'to': 0.024555401371788625,
 'me': 0.0058776703626907175,
 'because': 0.001526699833406824,
 'it': 0.01255080558680382,
 'one': 0.00287123415268136,
 'of': 0.015035745932906517,
 'the': 0.03297073359817933,
 'first': 0.0010006161202545964,
 'few': 0.0005925610287408761,
 'shows': 9.097426063128279e-05,
 'that': 0.011580870708378958,
 'i': 0.03403836176645154,
 'am': 0.00485215431063543,
 'aware': 3.664854574711102e-05,
 'focuses': 3.642380313204098e-06,
 'mainly': 3.0154259146121587e-05,
 'on': 0.006106869081335422,
 'work': 0.0010474245821519427,
 'not': 0.009818849857503108,
 'personal': 0.00010869792824042698,
 'life': 0.0010812212216112684,
 'said': 0.0009601469500512948,
 'cops': 1.7188935180184447e-05,
 '</s>': 0.055008963432966074,
 'if': 0.0027497258953819406,
 'think': 0.00185139866396

In [102]:
# Perplexity is calculated for a test string consisting of the first two sentences
# More than two produces a probability that gets approximated to 0, so the perplexity becomes undefined
test_string = ' '.join(test_sentences[0:2])
test_string

'<s> pretty much </s> <s> they could hit you and beat you and hit you </s>'

In [103]:
# Calculate N as the number of words
test_string_len = len(test_string.split(' '))
test_string_len

16

In [104]:
# Probability of the test sentence using unigrams
unigram_test_probability = get_sentence_probability_unigrams(test_string, unigram_probabilities)
unigram_test_probability

2.385751386228881e-38

In [105]:
# Perplexity for unigrams
get_perplexity(unigram_test_probability, test_string_len)

224.5941427265734

## Bigrams

In [52]:
# Retrieve bigram probabilities
bigram_file = open('BAC_G02_bigrams.json')
bigram_probabilities = json.load(bigram_file)
bigram_probabilities

{'<s> this': 0.011676417415928361,
 'this cop': 3.7241736109084065e-05,
 'cop show': 7.400928480118414e-05,
 'show is': 0.004535965791137687,
 'is interesting': 0.0005933460658153159,
 'interesting to': 0.0073409486623194214,
 'to me': 0.01740443698668777,
 'me because': 0.0038052371605280444,
 'because to': 0.0005019162226638543,
 'me it': 0.004974619096614488,
 'it is': 0.19878712604968282,
 'is one': 0.006469713552988384,
 'one of': 0.11421437242340837,
 'of the': 0.16250983100953173,
 'the first': 0.011679637203041536,
 'first few': 0.002274709473634585,
 'few shows': 0.00011303205804465304,
 'shows that': 0.0032197827624302192,
 'that i': 0.10647069046739563,
 'i am': 0.12434010588173801,
 'am aware': 0.000529732886534038,
 'aware of': 0.008784597272572647,
 'of that': 0.007385224361756098,
 'that focuses': 2.1235349704297755e-05,
 'focuses mainly': 2.029879831113998e-05,
 'mainly on': 0.000297653885386532,
 'on the': 0.1799518614229409,
 'the cop': 9.649322942836884e-05,
 'cop wo

In [106]:
# Probability of the test sentence using bigrams
bigram_test_probability = get_sentence_probability_bigrams(test_string, bigram_probabilities, unigram_frequencies)
bigram_test_probability

1.5980359966407296e-43

In [107]:
# Perplexity for bigrams
get_perplexity(bigram_test_probability, test_string_len)

472.907106297997

## Trigrams

In [88]:
# Retrieve trigram probabilities
trigram_file = open('BAC_G02_trigrams.json')
trigram_probabilities = json.load(trigram_file)
trigram_probabilities

{'<s> this cop': 1.0486577181208053e-05,
 'this cop show': 6.7762154836523805e-06,
 'cop show is': 1.0164839819065852e-05,
 'show is interesting': 1.3482358334119805e-05,
 'is interesting to': 0.000930809804529941,
 'interesting to me': 0.0006722417919277205,
 'to me because': 0.0014941079809564542,
 'me because to': 1.0029352571860311e-05,
 'because to me': 0.00015912784398699892,
 'to me it': 0.0027406048464248332,
 'me it is': 0.007134914134654441,
 'it is one': 0.0033608794227160525,
 'is one of': 0.026577666384245403,
 'one of the': 0.08421194444070217,
 'of the first': 0.00292845783582713,
 'the first few': 0.0022368987796123475,
 'first few shows': 1.0132465093657752e-05,
 'few shows that': 3.049245311785333e-05,
 'shows that i': 0.0003073275672827853,
 'that i am': 0.057023838694997726,
 'i am aware': 0.0005490168384147766,
 'am aware of': 0.0007205658979502776,
 'aware of that': 0.00022166470191135426,
 'of that focuses': 6.418052698630708e-06,
 'that focuses mainly': 6.776192

In [108]:
# Probability of the test sentence using trigrams
trigram_test_probability = get_sentence_probability_trigrams(test_string,trigram_probabilities,bigram_frequencies,vocab_len)
trigram_test_probability

1.557669575053562e-55

In [109]:
# Perplexity for trigrams
get_perplexity(trigram_test_probability, test_string_len)

2663.6078823860444

# Sentence Generation

## Unigrams

In [154]:
def get_next_word_unigrams(unigram_probabilities):
    """
    Returns a word using the probability distribution obtained from the training data
    """
    words = list(unigram_probabilities.keys())
    probabilities = list(unigram_probabilities.values())
    return np.random.choice(words, p=probabilities)

In [155]:
def generate_sentence_unigrams(unigram_probabilities, length):
    """
    Returns a sentence of length given by parameter
    For each word, selects a random word using the unigram probabilities
    """
    sentence_unigrams = []
    for i in range(length):
        sentence_unigrams.append(get_next_word_unigrams(unigram_probabilities))
    return ' '.join(sentence_unigrams)

In [156]:
# Example sentence using unigrams
generate_sentence_unigrams(unigram_probabilities, 15)

'and <s> nbsp hey extremists not picture session <s> drive can really been right fallen'

## Bigrams

In [157]:
def get_next_word_bigrams(bigram_probabilities,unigram_frequencies, previous_word):
    """
    Returns a bigram based on the bigram probabilities obtained from the training data and Laplace smoothing
    * Selects the bigram from a subset of bigrams that start with the word given by parameter
    """
    vocab_len = len(unigram_frequencies)
    bigrams = []
    probabilities = []
    # Iteratively creates the row of the Laplace smoothing matrix for a given term
    # That is, generates all possible bigrams (previous_word, X) where X is a term in the training data
    # and returns its probability estimate using Laplace smoothing
    for i in range(vocab_len):
        bigram = previous_word + ' ' + vocab_array[i]
        bigrams.append(bigram)
        # Case 1: the bigram appears in the training data (and its probability is already calculated)
        if bigram in bigram_probabilities:
            probabilities.append(bigram_probabilities[bigram])
        # Case 2: the bigram does not appear in the training data (happens due to Laplace smoothing)
        else:
            # The probability for the bigram (w1w2) is estimated as 1/(count(w1)+V)
            probabilities.append(1/(unigram_frequencies[previous_word]+vocab_len))
    return np.random.choice(bigrams, p=probabilities)

In [158]:
def generate_sentence_bigrams(bigram_probabilities, unigram_frequencies, initial_word, length):
    """
    Iteratively prints a sentence of a given length and using an initial word
    Uses the bigram probabilities to select the following word
    """
    sentence = initial_word
    for i in range(length):
        # Obtains a bigram using the last word of the sentence. Keeps the last word of the bigram
        next_word = get_next_word_bigrams(bigram_probabilities, unigram_frequencies, sentence.split(' ')[-1]).split(' ')[-1]
        # Adds the word to the sentence
        sentence += f' {next_word}'
        print(sentence)

In [159]:
# Example sentence using bigrams
generate_sentence_bigrams(bigram_probabilities, unigram_frequencies, 'i', 10)

i do
i do not
i do not many
i do not many abdurahman
i do not many abdurahman unslept
i do not many abdurahman unslept quickinspirations
i do not many abdurahman unslept quickinspirations kunci
i do not many abdurahman unslept quickinspirations kunci yogathon
i do not many abdurahman unslept quickinspirations kunci yogathon arrogance
i do not many abdurahman unslept quickinspirations kunci yogathon arrogance stooopid


## Trigrams

In [171]:
def get_next_word_trigrams(trigram_probabilities,bigram_frequencies, training_vocab_len, previous_bigram):
    """
    Returns a trigram based on the trigram probabilities obtained from the training data and Laplace smoothing
    * Selects the trigram from a subset of trigrams that start with the bigram given by parameter
    """
    trigrams = []
    probabilities = []
    # Iteratively creates the row of the Laplace smoothing matrix for two given terms
    # That is, generates all possible trigrams (previous_bigram[0], previous_bigram[1], X)
    # where X is a term in the training data, and returns its probability estimate using Laplace smoothing
    for i in range(training_vocab_len):
        trigram = previous_bigram + ' ' + vocab_array[i]
        trigrams.append(trigram)
        # Case 1: the trigram appears in the training data (and its probability is already calculated)
        if trigram in trigram_probabilities:
            probabilities.append(trigram_probabilities[trigram])
        # Case 2: the trigram does not appear in the training data (happens due to Laplace smoothing)
        else:
            # The probability for the trigram (w1w2w3) is estimated as 1/(count(w1w2)+V)
            probabilities.append(1/(bigram_frequencies[previous_bigram]+vocab_len))
    return np.random.choice(trigrams, p=probabilities)

In [172]:
def generate_sentence_trigrams(trigram_probabilities, bigram_frequencies, vocab_len, initial_words, length):
    """
    Iteratively prints a sentence of a given length and using an initial bigram
    Uses the trigram probabilities to select the following word
    """
    sentence = initial_words
    for i in range(length):
        sentence_list = sentence.split(' ')
        last_bigram = sentence_list[-2] + ' ' + sentence_list[-1]
        # Selects a trigram using the distribution estimated from the training data and Laplace smoothing
        # Keeps the last word of the trigram
        next_word = get_next_word_trigrams(trigram_probabilities, bigram_frequencies, vocab_len, last_bigram).split(' ')[-1]
        # Adds the word to the sentence
        sentence += f' {next_word}'
        print(sentence)

In [184]:
# Example sentence using trigrams
# Due to the computational cost of calculating the probability estimates, a sentence of only four words was generated
generate_sentence_trigrams(trigram_probabilities, bigram_frequencies, vocab_len, 'i want', 2)

i want to
i want to bananna
